In [1]:
import pandas as pd
import os
from tqdm.autonotebook import tqdm

/usr/local/lib/python3.5/dist-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [2]:
df = pd.read_json("~/datasets/prescriptionevents/tokenized/prescriptionevents.json", orient = "records", dtype="object")

In [3]:
df.head()

,ENDDATE,HADM_ID,ICUSTAY_ID,ROW_ID,STARTDATE,SUBJECT_ID,prescriptionevents
0,2175-06-12 00:00:00,107064,NaN,2214776,2175-06-11 00:00:00,6,"[MAIN, Tacrolimus, Tacrolimus, DRUG_CD-TACR1, ..."
1,2175-06-12 00:00:00,107064,NaN,2214775,2175-06-11 00:00:00,6,"[MAIN, Warfarin, Warfarin, DRUG_CD-WARF5, GSN-..."
2,2175-06-12 00:00:00,107064,NaN,2215524,2175-06-11 00:00:00,6,"[MAIN, Heparin, Sodium, NaN, DRUG_CD-HEPAPREMI..."
3,2175-06-12 00:00:00,107064,NaN,2216265,2175-06-11 00:00:00,6,"[BASE, D5W, NaN, DRUG_CD-HEPBASE, NaN, NaN, IV..."
4,2175-06-12 00:00:00,107064,NaN,2214773,2175-06-11 00:00:00,6,"[MAIN, Furosemide, Furosemide, DRUG_CD-FURO20,..."


In [4]:
df = df.rename({"STARTDATE":"STORETIME"}, axis='columns')

In [5]:
df.info(verbose = True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4156450 entries, 0 to 4156449
Data columns (total 7 columns):
ENDDATE               4156450 non-null object
HADM_ID               4156450 non-null object
ICUSTAY_ID            4156450 non-null object
ROW_ID                4156450 non-null object
STORETIME             4156450 non-null object
SUBJECT_ID            4156450 non-null object
prescriptionevents    4156450 non-null object
dtypes: object(7)
memory usage: 222.0+ MB


In [6]:
df_ICU = pd.read_csv("~/datasets/raw/ICUSTAYS.csv", dtype = "object")

In [7]:
df_ICU.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS
0,365,268,110404,280836,carevue,MICU,MICU,52,52,2198-02-14 23:27:38,2198-02-18 05:26:11,3.249
1,366,269,106296,206613,carevue,MICU,MICU,52,52,2170-11-05 11:05:29,2170-11-08 17:46:57,3.2788
2,367,270,188028,220345,carevue,CCU,CCU,57,57,2128-06-24 15:05:20,2128-06-27 12:32:29,2.8939
3,368,271,173727,249196,carevue,MICU,SICU,52,23,2120-08-07 23:12:42,2120-08-10 00:39:04,2.06
4,369,272,164716,210407,carevue,CCU,CCU,57,57,2186-12-25 21:08:04,2186-12-27 12:01:13,1.6202


In [8]:
df_ICU.info(verbose = True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61532 entries, 0 to 61531
Data columns (total 12 columns):
ROW_ID            61532 non-null object
SUBJECT_ID        61532 non-null object
HADM_ID           61532 non-null object
ICUSTAY_ID        61532 non-null object
DBSOURCE          61532 non-null object
FIRST_CAREUNIT    61532 non-null object
LAST_CAREUNIT     61532 non-null object
FIRST_WARDID      61532 non-null object
LAST_WARDID       61532 non-null object
INTIME            61532 non-null object
OUTTIME           61522 non-null object
LOS               61522 non-null object
dtypes: object(12)
memory usage: 5.6+ MB


In [9]:
# Selecting list of HADM_ID with single ICUSTAY
df_single_ICU = df_ICU.groupby(by=['HADM_ID'], as_index=False).agg({'ICUSTAY_ID': 'count'})
df_single_ICU  = df_single_ICU[df_single_ICU['ICUSTAY_ID']==1]['HADM_ID']

In [10]:
#number of HADM_ID with single ICUSTAY
len(df_single_ICU)

54526

In [11]:
# Dropping HADM_ID with mutiple ICUSTAYS
df = df[df['HADM_ID'].isin(df_single_ICU)].reset_index(drop = True)

In [12]:
df.info(verbose = True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3535521 entries, 0 to 3535520
Data columns (total 7 columns):
ENDDATE               3535521 non-null object
HADM_ID               3535521 non-null object
ICUSTAY_ID            3535521 non-null object
ROW_ID                3535521 non-null object
STORETIME             3535521 non-null object
SUBJECT_ID            3535521 non-null object
prescriptionevents    3535521 non-null object
dtypes: object(7)
memory usage: 188.8+ MB


In [13]:
#Mapping ICU INTIME based on ICUSTAY_ID
df = df.merge(df_ICU[['ICUSTAY_ID','INTIME']], how='left',left_on='ICUSTAY_ID', right_on='ICUSTAY_ID')

In [14]:
df = df.rename(columns={'INTIME':'ICU_INTIME'})

In [15]:
#Mapping ICU INTIME based on HADM_ID as every HADM_ID has only one ICUSTAY
df = df.merge(df_ICU[['HADM_ID','INTIME']], how='left',left_on='HADM_ID', right_on='HADM_ID')

In [16]:
df.head()

,ENDDATE,HADM_ID,ICUSTAY_ID,ROW_ID,STORETIME,SUBJECT_ID,prescriptionevents,ICU_INTIME,INTIME
0,2175-06-12 00:00:00,107064,NaN,2214776,2175-06-11 00:00:00,6,"[MAIN, Tacrolimus, Tacrolimus, DRUG_CD-TACR1, ...",NaN,2175-05-30 21:30:54
1,2175-06-12 00:00:00,107064,NaN,2214775,2175-06-11 00:00:00,6,"[MAIN, Warfarin, Warfarin, DRUG_CD-WARF5, GSN-...",NaN,2175-05-30 21:30:54
2,2175-06-12 00:00:00,107064,NaN,2215524,2175-06-11 00:00:00,6,"[MAIN, Heparin, Sodium, NaN, DRUG_CD-HEPAPREMI...",NaN,2175-05-30 21:30:54
3,2175-06-12 00:00:00,107064,NaN,2216265,2175-06-11 00:00:00,6,"[BASE, D5W, NaN, DRUG_CD-HEPBASE, NaN, NaN, IV...",NaN,2175-05-30 21:30:54
4,2175-06-12 00:00:00,107064,NaN,2214773,2175-06-11 00:00:00,6,"[MAIN, Furosemide, Furosemide, DRUG_CD-FURO20,...",NaN,2175-05-30 21:30:54


In [17]:
df = df.fillna('NaN')

In [18]:
# replacing updating ICU_INTIME for records with missing ICUSTAYID but have HADM_ID
df.loc[(df['HADM_ID']!='NaN') & (df['ICUSTAY_ID']=='NaN'),'ICU_INTIME'] = df['INTIME']

In [19]:
df = df.drop(labels=['ENDDATE','INTIME'], axis = 'columns')

In [20]:
df['STORETIME'] = pd.to_datetime(df['STORETIME'], errors='coerce')
df['ICU_INTIME'] = pd.to_datetime(df['ICU_INTIME'], errors='coerce')

In [21]:
%%time
df['icu_tdelta'] = df['STORETIME'] - df['ICU_INTIME']
df['icu_tdelta'] = df['icu_tdelta'].map(lambda x: x.total_seconds())

CPU times: user 42.1 s, sys: 700 ms, total: 42.8 s
Wall time: 42.8 s


In [22]:
df = df.drop(labels='ROW_ID', axis = 'columns')

In [23]:
df['STORETIME'] = df['STORETIME'].map(lambda x: str(x))

In [24]:
df.head()

,HADM_ID,ICUSTAY_ID,STORETIME,SUBJECT_ID,prescriptionevents,ICU_INTIME,icu_tdelta
0,107064,NaN,2175-06-11 00:00:00,6,"[MAIN, Tacrolimus, Tacrolimus, DRUG_CD-TACR1, ...",2175-05-30 21:30:54,959346.0
1,107064,NaN,2175-06-11 00:00:00,6,"[MAIN, Warfarin, Warfarin, DRUG_CD-WARF5, GSN-...",2175-05-30 21:30:54,959346.0
2,107064,NaN,2175-06-11 00:00:00,6,"[MAIN, Heparin, Sodium, NaN, DRUG_CD-HEPAPREMI...",2175-05-30 21:30:54,959346.0
3,107064,NaN,2175-06-11 00:00:00,6,"[BASE, D5W, NaN, DRUG_CD-HEPBASE, NaN, NaN, IV...",2175-05-30 21:30:54,959346.0
4,107064,NaN,2175-06-11 00:00:00,6,"[MAIN, Furosemide, Furosemide, DRUG_CD-FURO20,...",2175-05-30 21:30:54,959346.0


In [25]:
sec_24hrs = 24*60*60 #total seconds in 24 hrs
sec_48hrs = 48*60*60 #total seconds in 48 hrs

In [26]:
print("Total Records: " + str(len(df)))
print("Total Records Prior to 24hrs in ICU: " +str(len(df[df['icu_tdelta']<sec_24hrs])))
print("Total Records Prior to 48hrs in ICU: " +str(len(df[df['icu_tdelta']<sec_48hrs])))

Total Records: 3535521
Total Records Prior to 24hrs in ICU: 1874578
Total Records Prior to 48hrs in ICU: 2212109


In [27]:
# create output path
mypath_output = "/home/jupyter/datasets/data_before_24hrs_icu/"
import os
os.makedirs(mypath_output, exist_ok=True)

In [28]:
# extract data recorded in ans prior to first 24hrs of ICU stay
df[df['icu_tdelta']<sec_24hrs].to_json(mypath_output+"prescriptionevents.json", orient = 'records')

In [29]:
# create output path
mypath_output = "/home/jupyter/datasets/data_before_48hrs_icu/"
import os
os.makedirs(mypath_output, exist_ok=True)

In [30]:
# extract data recorded in ans prior to first 48hrs of ICU stay
df[df['icu_tdelta']<sec_48hrs].to_json(mypath_output+"prescriptionevents.json", orient = 'records')